In [ ]:
import random
import math
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

---

### Train Data

In [ ]:
skip1 = []

for i in range(1, 514053):
    skip1.append(i)

for i in range(2067401, 2390492): #1832886
    skip1.append(i)

In [ ]:
x_tr = pd.read_csv('train.csv', skiprows = skip1)
x_tr

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,85,13.407566,0.000145,-0.009279,-0.014769,-0.021924,-0.023899,-1,7.016810,5.069986,...,NaN,2.553861,4.916348,1.242548,3.938603,2.021285,3.457164,2.695782,4.503388,514052
1,85,0.000000,-0.005933,-0.009812,-0.028385,-0.062846,-0.049266,-1,0.016471,-0.658689,...,NaN,5.460800,2.560419,6.910745,5.178953,8.599560,4.032265,6.834335,3.001478,514053
2,85,0.644619,0.013643,0.008948,0.027272,0.051507,0.036758,-1,-3.172026,-3.093182,...,NaN,7.019334,7.871258,6.345565,9.437935,8.060132,7.829917,7.285557,7.238598,514054
3,85,0.113759,-0.012292,-0.007364,0.007116,0.048304,0.033099,-1,-3.172026,-3.093182,...,NaN,5.234132,16.552604,2.601202,12.545645,3.806290,11.278648,4.226967,12.536524,514055
4,85,1.003626,-0.018024,-0.009781,-0.022747,-0.078673,-0.056691,-1,-1.051912,-1.819816,...,NaN,2.288103,0.827409,2.354786,1.849218,3.291306,1.348731,2.648102,0.863261,514056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553343,439,0.000000,-0.000024,-0.000024,0.000190,-0.003013,-0.002756,-1,0.092671,0.955919,...,-3.142904,-1.028317,-1.846409,-0.718009,-2.360138,-0.803080,-1.194592,-1.027671,-1.545331,2067395
1553344,439,0.000000,0.000179,0.000179,-0.000410,0.000984,0.001841,1,-0.706975,0.083686,...,-2.643917,-1.622582,-0.047453,-1.842869,-2.490789,-2.543397,-0.817713,-1.638932,0.347771,2067396
1553345,439,0.000000,-0.000100,-0.000100,-0.002284,-0.007031,-0.004743,1,1.304405,0.502202,...,-2.076216,2.469565,1.495541,1.192580,0.314879,4.001326,2.556813,4.207967,2.933677,2067397
1553346,439,0.000000,0.000305,0.000305,0.000608,-0.003047,-0.002915,1,0.029028,1.766397,...,-2.485007,1.400353,2.862522,-0.491200,-1.576946,-0.246677,-0.187300,0.943114,1.739998,2067398


---

### Test Data

In [ ]:
x_ts = pd.read_csv('train.csv', skiprows = [i for i in range(1, 2067401)])
x_ts

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,440,0.000000,-0.006547,-0.005719,-0.010282,-0.021065,-0.017211,1,-1.199292,-0.821106,...,NaN,-1.789993,1.033497,-1.283115,2.152617,-2.153074,1.608898,-1.805725,1.618956,2067400
1,440,0.415759,-0.000140,0.003201,0.020718,0.069251,0.048473,-1,0.402453,-1.304766,...,NaN,1.025831,-0.933187,2.351714,1.637792,2.802581,0.703759,1.204082,-0.835363,2067401
2,440,0.051447,0.003028,0.002853,0.001346,0.003306,0.004821,1,0.165805,1.398739,...,NaN,-1.992154,0.265553,-1.127521,2.567383,-1.869028,1.921513,-1.911805,1.091962,2067402
3,440,0.364965,-0.021950,-0.016466,-0.039547,-0.059556,-0.033564,-1,1.761655,-0.592444,...,NaN,3.846277,1.578965,3.157499,1.851599,4.327416,1.344662,4.025867,1.285276,2067403
4,440,0.330940,0.016254,0.026992,0.052505,0.074352,0.060841,-1,-3.172026,-3.093182,...,NaN,0.025794,1.934294,0.008607,2.763888,-0.034986,2.007679,-0.314655,1.117603,2067404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323086,499,0.000000,0.000142,0.000142,0.005829,0.020342,0.015396,1,-1.649365,-1.169996,...,-1.896874,-1.260055,1.947725,-1.994399,-1.685163,-2.866165,-0.216130,-1.892048,0.901585,2390486
323087,499,0.000000,0.000012,0.000012,-0.000935,-0.006326,-0.004718,1,2.432943,5.284504,...,-0.936553,1.064936,3.119762,-0.419796,-0.208975,-0.146749,0.730166,0.648452,2.068737,2390487
323088,499,0.000000,0.000499,0.000499,0.007605,0.024907,0.016591,1,-0.622475,-0.963682,...,-2.956745,-0.640334,-2.279663,-0.950259,-4.388417,-1.669922,-3.288939,-1.336142,-2.814239,2390488
323089,499,0.283405,-0.000156,-0.000156,-0.001375,-0.003702,-0.002004,-1,-1.463757,-1.107228,...,-2.035894,-1.780962,0.881246,-2.202140,-1.912601,-3.341684,-0.571188,-2.185795,0.627452,2390489


---

### Lable Creation

In [ ]:
y_tr = np.array([(x_tr['resp'] > 0).astype('int')])
y_ts = np.array([(x_ts['resp'] > 0).astype('int')])

y_tr = y_tr.reshape(-1)
y_ts = y_ts.reshape(-1)

print(y_tr.shape)
print(y_ts.shape)

(1553348,)
(323091,)


---

In [ ]:
# Droping all the resp columns, ts_id, feature_0 & weight columns
x_tr.drop(['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id', 'feature_0', 'weight'], axis=1, inplace = True)
x_ts.drop(['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id', 'feature_0', 'weight'], axis=1, inplace = True)

display(x_tr.head(1))
display(x_ts.head(1))

,date,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
0,85,7.01681,5.069986,-0.076187,-0.073584,-0.012208,-0.020858,NaN,NaN,4.340084,...,NaN,NaN,2.553861,4.916348,1.242548,3.938603,2.021285,3.457164,2.695782,4.503388


,date,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
0,440,-1.199292,-0.821106,0.17372,0.234414,0.127096,0.178566,NaN,NaN,-0.727608,...,NaN,NaN,-1.789993,1.033497,-1.283115,2.152617,-2.153074,1.608898,-1.805725,1.618956


---

## Creating GroupTimeSeriesSplit Cross-Validation indicies

In [ ]:
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243

class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

---

## Custom Model
### (In this model the metadata will contain the calibrated probablities values of outcome 1)

In [ ]:
class CustomModel():
    
    """
    The model will split the input dataframe into two equal parts during training. One part will be used for training
    n_estimators and the second part will be used to generate metadata by predicting the outcome using trained
    n_estimators. Then the model will again be trained on this metadata to give us the final estimator.
    
    -->base_estimator: The base estimator from which the ensemble is built.
    -->n_estimators: Number of estimators to be used.
    -->sample_ratio: Fraction of rows to be sampled with repacement.
    -->group_split: Split the input data while keeping the group in consideration i.e. the data will be divided in
                 such a way that the same group of the mentioned column will not appear in both samples.
    -->group: Name of the column using which the group_split will be performed.
    -->keep_group: To keep the column mentioned in the group or not after spliting
    """

    def __init__(self, base_estimator = DecisionTreeClassifier(), n_estimators = 50, sample_ratio = 0.3, \
                 group_split = False, group = None, keep_group = False, random_state = None):

        super(CustomModel, self).__init__()
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.sample_ratio = sample_ratio
        self.group_split = group_split
        self.group = group
        self.keep_group = keep_group
        self.random_state = random_state
        self.k_models = []
        self.meta_model = None
        self.metadata = []
        self.pred_metadata = []
        self.fit_status = False
        self.md_status = False

    def split(self, df, y):
        
        """
        The function will take dataframe and corresponding labels as input and
        split the dataframe in two parts to return both splits of dataframe and labels.
        """

        pd.options.mode.chained_assignment = None

        if (self.group_split == False) and (self.group != None):

            raise Exception('group cannot have value when group_split = False')

        if self.group_split == True:

            if self.group == None:

                raise Exception('when group_split = True, group cannot be None')

            if isinstance(self.group, str) == False:

                raise Exception('group can only have string as a value, the string should be name of the column of dataframe using which you want to do group_split')

        if self.group_split == True:

            len = df.shape[0]
            x = math.floor(len/2)

            x_ = df[self.group][x]

            grp = df[df[self.group] == x_]

            idx = grp.index[0]

            d1 = df.iloc[:idx, :]
            d2 = df.iloc[idx:, :]

            idx_1 = d1.index
            idx_2 = d2.index

            y1 = y[idx_1]
            y2 = y[idx_2]

            d1.reset_index(inplace = True)
            d1.drop(['index'], axis = 1, inplace = True)

            d2.reset_index(inplace = True)
            d2.drop(['index'], axis = 1, inplace = True)

            return d1, d2, y1, y2

        else:

            len = df.shape[0]
            idx = math.floor(len/2)

            d1 = df.iloc[:idx, :]
            d2 = df.iloc[idx:, :]

            idx_1 = d1.index
            idx_2 = d2.index

            y1 = y[idx_1]
            y2 = y[idx_2]

            d1.reset_index(inplace = True)
            d1.drop(['index'], axis = 1, inplace = True)

            d2.reset_index(inplace = True)
            d2.drop(['index'], axis = 1, inplace = True)

            return d1, d2, y1, y2

    def sample(self, d1, rand = None):
        
        """
        The function will take a dataframe and return row sampling with replacement.
        """
        
        d1_ = d1.sample(frac = self.sample_ratio, replace = True, axis = 0, random_state = rand).index
    
        return d1_

    def metamodel(self, d2, y2):
        
        """
        The function will predict the calibrated probablities of outcome 1
        of 2nd half of the input dataframe, using the n_estimators trained
        on 1st half of the input dataframe. Thus greating a metadata.
        
        Using this metadata a model will be again trained and returned
        """
        
        self.metadata = []
        for i in self.k_models:

            self.metadata.append(i.predict_proba(d2)[:, 1])

        self.metadata = np.array(self.metadata).T
        
        self.meta_model = CalibratedClassifierCV(self.base_estimator, method = 'sigmoid')
        
        self.meta_model.fit(self.metadata, y2)

    def fit(self, df, y):
        
        """
        The function will take a Pandas Dataframe as train data and nparray as lables
        and train the model.
        """

        if (self.keep_group == True) and (self.group_split == False):

            raise Exception('keep_group cannot be True if group_split is False')

        d1, d2, y1, y2 = self.split(df, y)

        if (self.keep_group == False) and (self.group_split == True):

            d1.drop([self.group], axis = 1, inplace = True)
            d2.drop([self.group], axis = 1, inplace = True)
        
        if self.random_state != None:
            random.seed(self.random_state)
            
        for i in range(self.n_estimators):
            
            if self.random_state == None:       
                smpl_idx = self.sample(d1)
                
            else:
                smpl_idx = self.sample(d1, random.randint(0, self.n_estimators*10))

            d1_ = d1.iloc[smpl_idx, :]
            y1_ = y1[smpl_idx]
            
            cln = clone(self.base_estimator)

            model = CalibratedClassifierCV(cln, method = 'sigmoid')
            model.fit(d1_, y1_)

            self.k_models.append(model)
        
        self.metamodel(d2, y2)

        self.fit_status = True
        self.md_status = True

    def get_params(self, deep = False):
        
        """
        The function will return a dictionary containing parameters of the model,
        with parameter names as keys and there corresponding values as values.
        """

        params = {'base_estimator' : self.base_estimator, \
                  'n_estimators' : self.n_estimators, \
                  'sample_ratio' : self.sample_ratio, \
                  'group_split' : self.group_split, \
                  'group' : self.group, \
                  'keep_group' : self.keep_group, \
                  'random_state' : self.random_state}

        return params

    def set_params(self, **params):
        
        """
        The function take a dictionary as input,
        with parameter names as keys and there corresponding values as values
        and set the parameters of the model accordingly.
        """
        
        if not params:

            raise Exception('You need to provide a dictnory of parameters')

        if isinstance(params, dict) == False:

            raise Exception('params need to be a dictnory')

        for i, j in params.items():

            setattr(self, i, j)

        return self
    
    def get_metadata(self):
        
        """
        The function will return the metadata of the model.
        """
        
        if self.fit_status == False:

            raise Exception('There is no metadata if model is not trained, please fit the model first.')

        if self.md_status == False:

            raise Exception('The loaded model did not have metadata saved')
            
        return self.metadata

    def predict(self, x):
        
        """
        Given a quary dataframe the function will return the predicted outcome.
        """

        if self.fit_status == False:

            raise Exception('Predict cannot be used if model is not trained, please fit the model first.')

        if self.keep_group == False:
          
            x_ = x.drop(self.group, axis = 1)

        self.pred_metadata = []

        for i in self.k_models:

            self.pred_metadata.append(i.predict(x_))

        self.pred_metadata = np.array(self.pred_metadata).T       

        return self.meta_model.predict(self.pred_metadata)
    
    def save_model(self, _, save_metadata = False):
        
        """
        Upon function call function will save all the necessary parameters and variables of 
        the model in a pickel file at the given location.
        """
        
        if isinstance(_, str) == False:
            
            raise Exception('location needs to be a string i.e. address of where to save the model with file name')
        
        if self.fit_status == True:

            if save_metadata == True:

                var_ = {'base_estimator' : self.base_estimator, \
                        'n_estimators' : self.n_estimators, \
                        'sample_ratio' : self.sample_ratio, \
                        'group_split' : self.group_split, \
                        'group' : self.group, \
                        'keep_group' : self.keep_group, \
                        'random_state' : self.random_state, \
                        'k_models' : self.k_models, \
                        'meta_model' : self.meta_model, \
                        'metadata' : self.metadata, \
                        'pred_metadata' : self.pred_metadata, \
                        'fit_status' : self.fit_status, \
                        'md_status' : self.md_status}

            else:

                var_ = {'base_estimator' : self.base_estimator, \
                        'n_estimators' : self.n_estimators, \
                        'sample_ratio' : self.sample_ratio, \
                        'group_split' : self.group_split, \
                        'group' : self.group, \
                        'keep_group' : self.keep_group, \
                        'random_state' : self.random_state, \
                        'k_models' : self.k_models, \
                        'meta_model' : self.meta_model, \
                        'pred_metadata' : self.pred_metadata, \
                        'fit_status' : self.fit_status, \
                        'md_status' : False}
            
        with open(_ + '.pkl', 'wb') as file:
            pickle.dump(var_, file)
            
        return self
            
    def load_model(self, _):
        
        """
        Upon function call the function will load the saved model form the given location.
        """
        
        if isinstance(_, str) == False:
            
            raise Exception('location needs to be a string i.e. address from where to load the model')
        
        with open(_, 'rb') as file:
            var_ = pickle.load(file)
            
        for i, j in var_.items():

            setattr(self, i, j)

        return self

---

## Missing value imputation

In [ ]:
fillna_tr_mean = x_tr.mean()

with open('fillna.pkl', 'wb') as file:
    pickle.dump(fillna_tr_mean, file)

In [ ]:
x_tr.fillna(fillna_tr_mean, inplace = True)
x_ts.fillna(fillna_tr_mean, inplace = True)

---

## Cross-Validation

In [ ]:
gtss = GroupTimeSeriesSplit(n_splits = 5)
gtss_cv = gtss.split(x_tr, groups = x_tr['date'])

In [ ]:
%%time
cm = CustomModel(base_estimator = DecisionTreeClassifier(max_depth = 7, max_features = 'sqrt', random_state = 64), \
                 group_split = True, group = 'date', random_state = 64)

params = {'n_estimators' : [100, 200, 300], \
          'sample_ratio': [0.05, 0.1, 0.25, 0.5]}

cv = RandomizedSearchCV(cm, params, scoring = 'f1', cv = gtss_cv, verbose = 1, \
                        return_train_score = True, n_jobs = 8)
cv.fit(x_tr, y_tr)

results = pd.DataFrame.from_dict(cv.cv_results_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
CPU times: user 17min 17s, sys: 5min 51s, total: 23min 9s
Wall time: 5h 5min 24s


In [ ]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
results[['rank_test_score', 'mean_train_score', 'mean_test_score', \
         'split4_train_score', 'split4_test_score', 'params']]

,rank_test_score,mean_train_score,mean_test_score,split4_train_score,split4_test_score,params
0,2,0.562672,0.558289,0.596001,0.583581,"{'sample_ratio': 0.1, 'n_estimators': 300}"
1,6,0.528837,0.529980,0.637140,0.626012,"{'sample_ratio': 0.05, 'n_estimators': 100}"
2,5,0.537916,0.532183,0.434642,0.429608,"{'sample_ratio': 0.5, 'n_estimators': 200}"
3,3,0.561629,0.557190,0.636780,0.625533,"{'sample_ratio': 0.05, 'n_estimators': 200}"
4,7,0.506335,0.500950,0.428862,0.427772,"{'sample_ratio': 0.25, 'n_estimators': 100}"
5,10,0.494369,0.487907,0.590724,0.579877,"{'sample_ratio': 0.1, 'n_estimators': 200}"
6,4,0.550874,0.547476,0.594491,0.581614,"{'sample_ratio': 0.5, 'n_estimators': 100}"
7,8,0.498108,0.498264,0.388167,0.392358,"{'sample_ratio': 0.5, 'n_estimators': 300}"
8,9,0.487961,0.489292,0.582968,0.562762,"{'sample_ratio': 0.25, 'n_estimators': 200}"
9,1,0.571303,0.567963,0.591865,0.580880,"{'sample_ratio': 0.1, 'n_estimators': 100}"


In [ ]:
cv.best_params_

{'sample_ratio': 0.1, 'n_estimators': 100}

---

## Modeling

In [ ]:
%%time
cm = CustomModel(base_estimator = DecisionTreeClassifier(max_depth = 7, max_features = 'sqrt', random_state = 64), \
                 group_split = True, group = 'date', random_state = 64)
cm.set_params(**cv.best_params_)
cm.fit(x_tr, y_tr)

CPU times: user 14min 9s, sys: 5min 30s, total: 19min 40s
Wall time: 11min 18s


In [ ]:
cm.get_params()

{'base_estimator': DecisionTreeClassifier(max_depth=7, max_features='sqrt', random_state=64),
 'n_estimators': 100,
 'sample_ratio': 0.1,
 'group_split': True,
 'group': 'date',
 'keep_group': False,
 'random_state': 64}

In [ ]:
y_tr_pred = cm.predict(x_tr)
y_ts_pred = cm.predict(x_ts)

In [ ]:
f1_tr = f1_score(y_tr, y_tr_pred)
f1_ts = f1_score(y_ts, y_ts_pred)

print('f1 Score Train:', f1_tr)
print('f1 Score Test:', f1_ts)

f1 Score Train: 0.612572771969852
f1 Score Test: 0.6066336477677781
